Heroes Of Pymoli Data Analysis

Observations:
(1) Item 'Final Critic' is listed under two different Item IDs.  In order to be the most accurate, input from the data owner could clarify the nature of this item (by name, it is the most popular purchase).  For this analysis, I have assumed that Item IDs are unique.
(2) Males are overwhelmingly the largest purchasing group by gender, both by number of purchases and amount of purchases.
(3) The age group of 20-24 year-olds spent more than 2.5 times more than the second-highest grossing group (15-19 year-olds).


In [1]:
#Import dependencies
import json
import pandas as pd
data=json.load(open('purchase_data.json'))
purchase_data_df=pd.DataFrame(data)

In [2]:
#Define function for formatting currency 
def currency(x):
    return "${:.2f}".format(x)
#Define function for formatting percent
def percent(x):
    return "{:.2f}%".format(x)

Player Count

In [3]:
#Calculate total players and display in DataFrame
players = purchase_data_df["SN"].nunique()
players_df = {"Total Players":[players]}
players_df = pd.DataFrame(data=players_df)
players_df.head()

,Total Players
0,573


Purchasing Analysis (Total)

In [4]:
#Calculate values based on unique items, format and display in DataFrame

unique_items = len(purchase_data_df["Item ID"].unique())
avg_purchase_price = purchase_data_df["Price"].mean()
total_purchases = purchase_data_df["Item ID"].count()
total_revenue = purchase_data_df["Price"].sum()

purchase_analysis = {
    "Unique Items":[unique_items],
    "Average Purchase Price":[avg_purchase_price],
    "Total Number of Purchases":[total_purchases],
    "Total Revenue":[total_revenue]
}

purchase_analysis_df = pd.DataFrame(data=purchase_analysis)
purchase_analysis_df["Average Purchase Price"] = purchase_analysis_df["Average Purchase Price"].apply(currency)
purchase_analysis_df["Total Revenue"] = purchase_analysis_df["Total Revenue"].apply(currency)
purchase_analysis_df.head()

,Average Purchase Price,Total Number of Purchases,Total Revenue,Unique Items
0,$2.93,780,$2286.33,183


Gender Demographics

In [5]:
#Calculate percent of players based on gender, format and display in DataFrame
unique_SN_df = purchase_data_df.drop_duplicates(["SN"])
male_count = unique_SN_df.Gender.value_counts()["Male"]
female_count = unique_SN_df.Gender.value_counts()["Female"]
other_count = unique_SN_df.Gender.value_counts()["Other / Non-Disclosed"]
percent_male = (male_count / players)*100
percent_female = (female_count / players)*100
percent_other = (other_count / players)*100

purchase_by_gender = {"Gender":[male_count,female_count,other_count],
                     "Percent of Players":[percent_male,percent_female,percent_other]}

purchase_by_gender_df = pd.DataFrame(data=purchase_by_gender,index=["Male","Female","Other"])
purchase_by_gender_df["Percent of Players"] = purchase_by_gender_df["Percent of Players"].apply(percent)
purchase_by_gender_df.head()

,Gender,Percent of Players
Male,465,81.15%
Female,100,17.45%
Other,8,1.40%


Purchasing Analysis (Gender)

In [6]:
#Calculate purchase data based on gender, format for display
purchase_count_gender = purchase_data_df[["Gender","Item ID"]]
purchase_count_gender = purchase_count_gender.groupby(["Gender"]).count()
purchase_count_gender = purchase_count_gender.rename(columns={"Item ID":"Purchase Count"})

amount_gender = purchase_data_df[["Gender","Price"]]
amount_gender = amount_gender.groupby(["Gender"]).sum()
amount_gender = amount_gender.rename(columns={"Price":"Total Purchase Value"})

gender_totals_df = purchase_count_gender.join(amount_gender,how="right")

average_price_array = []
normalized_total_array = []
for x in range(len(gender_totals_df)):
    average_purchase_price = gender_totals_df.iloc[x]["Total Purchase Value"]/gender_totals_df.iloc[x]["Purchase Count"]
    average_price_array.append(average_purchase_price)
    normalized_total = (gender_totals_df.iloc[x]["Total Purchase Value"]-gender_totals_df["Total Purchase Value"].min())/(gender_totals_df["Total Purchase Value"].max()-gender_totals_df["Total Purchase Value"].min())                                                        
    normalized_total_array.append(normalized_total)

gender_totals_df["Average Purchase Price"] = average_price_array
gender_totals_df["Normalized Totals"] = normalized_total_array
gender_totals_df["Total Purchase Value"] = gender_totals_df["Total Purchase Value"].apply(currency)
gender_totals_df["Average Purchase Price"] = gender_totals_df["Average Purchase Price"].apply(currency)
gender_totals_df.head()

,Purchase Count,Total Purchase Value,Average Purchase Price,Normalized Totals
Gender,,,,
Female,136,$382.91,$2.82,0.189509
Male,633,$1867.68,$2.95,1.000000
Other / Non-Disclosed,11,$35.74,$3.25,0.000000


In [7]:
#Set parameters for bins based on DataFrame values
max_age = purchase_data_df["Age"].max()
min_age = purchase_data_df["Age"].min()

num_of_bins = (int(((max_age-min_age)/4)))

bins = []
bins.append(min_age)
y = 10
for x in range(num_of_bins):
    bins.append(y-1)
    y = y+5

bin_labels = []
bin_labels.append("<10") 
z = 10
for x in range(num_of_bins): 
    bin_labels.append(""+str(z)+"-"+str(z+4)+"") 
    z = z+5
bin_labels = bin_labels[:-2] 
bin_labels.append(""+str(z-10)+"+") 

#Create DataFrame with bins, calculate correspoding purchase data and format

purchase_data_df["Age Range"] = pd.cut(purchase_data_df["Age"],bins,labels=bin_labels, include_lowest=True)
purchase_by_age = purchase_data_df.groupby(purchase_data_df["Age Range"])

Age Demographics

In [8]:
#Find, calculate, format and display purchase data based on age range bins
players_by_age = purchase_by_age[["Age Range","SN"]].count()
players_by_age = players_by_age.drop(columns={"Age Range"})
players_by_age = players_by_age.rename(columns={"SN":"Total Count"})
age_percent_array = []
for x in range(len(players_by_age)):
    age_percent = (players_by_age.iloc[x]["Total Count"]/players)*100
    age_percent_array.append(age_percent)
players_by_age["Percentage of Players"] = age_percent_array
players_by_age["Percentage of Players"] = players_by_age["Percentage of Players"].apply(percent)
players_by_age.head(10)

,Total Count,Percentage of Players
Age Range,,
<10,28,4.89%
10-14,35,6.11%
15-19,133,23.21%
20-24,336,58.64%
25-29,125,21.82%
30-34,64,11.17%
35-39,42,7.33%
40-44,16,2.79%
45+,1,0.17%


Purchasing Analysis (Age)

In [9]:
#Find, calculate, format and display purchase data based on age range bins
items_by_age = purchase_by_age.count()
items_by_age = items_by_age.drop(columns={"Age","Gender","Item Name","SN","Price"})
items_by_age = items_by_age.rename(columns={"Item ID":"Item Count"})

amount_by_age = purchase_by_age["Price"].sum()
amount_by_age_df = pd.DataFrame(pd.Series(amount_by_age))
amount_by_age_df = amount_by_age_df.rename(columns={"Price":"Total Purchase Value"})

age_range_df = amount_by_age_df.join(items_by_age)
average_price_array = []
for x in range(len(age_range_df)):
    average_purchase_price = age_range_df.iloc[x]["Total Purchase Value"]/age_range_df.iloc[x]["Item Count"]
    average_price_array.append(average_purchase_price)
age_range_df["Average Purchase Price"] = average_price_array

normalized_total_array = []
for x in range(len(age_range_df)):
    normalized_total = (age_range_df.iloc[x]["Total Purchase Value"]-age_range_df["Total Purchase Value"].min())/(age_range_df["Total Purchase Value"].max()-age_range_df["Total Purchase Value"].min())                                                        
    normalized_total_array.append(normalized_total)
age_range_df["Normalized Totals"] = normalized_total_array
age_range_df["Total Purchase Value"] = age_range_df["Total Purchase Value"].apply(currency)
age_range_df["Average Purchase Price"] = age_range_df["Average Purchase Price"].apply(currency)
age_range_df.head(10)

,Total Purchase Value,Item Count,Average Purchase Price,Normalized Totals
Age Range,,,,
<10,$83.46,28,$2.98,0.082721
10-14,$96.95,35,$2.77,0.096542
15-19,$386.42,133,$2.91,0.393115
20-24,$978.77,336,$2.91,1.000000
25-29,$370.33,125,$2.96,0.376630
30-34,$197.25,64,$3.08,0.199303
35-39,$119.40,42,$2.84,0.119543
40-44,$51.03,16,$3.19,0.049495
45+,$2.72,1,$2.72,0.000000


Top Spenders

In [10]:
#Find, calculate, format and display purchase data based on unique users

SN_purchase = purchase_data_df[["SN", "Price"]]
SN_items = purchase_data_df[["SN", "Item ID"]]

SN_items_group = SN_items.groupby(["SN"])
SN_items_count = SN_items_group.count()
SN_items_count = SN_items_count.sort_values(["Item ID"],ascending=False)

SN_purchase_group = SN_purchase.groupby(["SN"])
SN_purchase_sum = SN_purchase_group.sum()
SN_purchase_sum = SN_purchase_sum.sort_values(["Price"],ascending=False)

SN_purchase_totals = SN_purchase_sum.join(SN_items_count,how="right")
SN_purchase_totals = SN_purchase_totals.sort_values(["Price"],ascending=False)

average_price_array = []
for x in range(len(SN_purchase_totals)):
    average_price = SN_purchase_totals.iloc[x]["Price"]/SN_purchase_totals.iloc[x]["Item ID"]
    average_price_array.append(average_price)
SN_purchase_totals["Average Price"] = average_price_array
SN_purchase_totals["Price"]=SN_purchase_totals["Price"].apply(currency)
SN_purchase_totals["Average Price"]=SN_purchase_totals["Average Price"].apply(currency)
SN_purchase_totals = SN_purchase_totals.rename(columns={"Price":"Total Purchase Value","Item ID":"Puchase Count","Average Price":"Average Purchase Price"})
SN_purchase_totals.head()

,Total Purchase Value,Puchase Count,Average Purchase Price
SN,,,
Undirrala66,$17.06,5,$3.41
Saedue76,$13.56,4,$3.39
Mindimnya67,$12.74,4,$3.18
Haellysu29,$12.73,3,$4.24
Eoda93,$11.58,3,$3.86


In [11]:
#Gather and calculate purchase data based on Item ID volume
item_totals_df = purchase_data_df[["Item ID","Item Name","Price"]]
item_df = purchase_data_df[["Item ID","Item Name"]]

item_ID_group = item_df.groupby(["Item ID"],as_index=False)
item_ID_count = item_ID_group.count()
item_ID_count = item_ID_count.rename(columns={"Item Name":"Item Count"})
item_ID_count = item_ID_count.sort_values(["Item Count"],ascending =False)

item_ID_count = item_ID_count.merge(item_totals_df,on="Item ID").drop_duplicates().reset_index()
item_ID_count = item_ID_count.sort_values(["Item Count"],ascending=False)

amount_spent_array = []
for x in range(len(item_ID_count)):
    amount_spent = item_ID_count.iloc[x]["Price"]*item_ID_count.iloc[x]["Item Count"]
    amount_spent_array.append(amount_spent)
item_ID_count["Total Purchase Value"] = amount_spent_array
item_ID_count = item_ID_count.drop(columns={"index"})
item_ID_count = item_ID_count.rename(columns={"Item Count":"Purchase Count","Price":"Item Price"})

In [12]:
#Sort Item ID DataFrame to find same data by greatest purchase amount
most_profit_df = item_ID_count.sort_values(["Total Purchase Value"],ascending=False)

Most Popular Items

In [13]:
#Format and display based on Item ID volume
#Display 6 items because there is a tie for 5th
item_ID_count["Item Price"] = item_ID_count["Item Price"].apply(currency)
item_ID_count["Total Purchase Value"] = item_ID_count["Total Purchase Value"].apply(currency)
item_ID_count.head(6)

,Item ID,Purchase Count,Item Name,Item Price,Total Purchase Value
0,39,11,"Betrayal, Whisper of Grieving Widows",$2.35,$25.85
1,84,11,Arcane Gem,$2.23,$24.53
2,31,9,Trickster,$2.07,$18.63
3,175,9,Woeful Adamantite Claymore,$1.24,$11.16
4,13,9,Serenity,$1.49,$13.41
5,34,9,Retribution Axe,$4.14,$37.26


Most Profitable Items

In [14]:
#Format and display based on purchase volume
most_profit_df["Item Price"] = most_profit_df["Item Price"].apply(currency)
most_profit_df["Total Purchase Value"] = most_profit_df["Total Purchase Value"].apply(currency)
most_profit_df.head()

,Item ID,Purchase Count,Item Name,Item Price,Total Purchase Value
5,34,9,Retribution Axe,$4.14,$37.26
13,115,7,Spectral Diamond Doomblade,$4.25,$29.75
39,32,6,Orenmir,$4.95,$29.70
24,103,6,Singed Scalpel,$4.87,$29.22
9,107,8,"Splitter, Foe Of Subtlety",$3.61,$28.88
